# decision trees; random forests
_COSC 101, Introduction to Computing I, 2021-12-06_

## Announcements
* Homework #10 due Thursday @ 5pm
* DEI reflection due today @ 11pm
* Attend faculty candidate research talks
    * 11:20am tomorrow and Wed, Dec 15
    * Earn 2 points of extra credit on final exam for each talk you attend (earnings capped at 4 points)

## Outline
* Decision trees
* scikit-learn
* Accuracy
* Random forests

## No warm-up — Happy last week of classes!

## Decision trees
* Similar to nested if-statements
* At each node in a decision tree, a decision is made (based on features of the data) and the appropriate branch is taken
* When there are no more branches, a final outcome (e.g., a label) is decided

### Activity
* Pictures of things
* Manually construct a decision tree to decide _does it fly?_
* Start with a single pile of pictures
* Select one meaningful characteristic that can be used to divide the pictures into two piles
    * For simplicitly: write down a question with a yes/no answer
* For each pile, select a meaningful characteristic that can be used to divide into two subpiles
    * Again, write down a question with a yes/no answer
    * The question can be different for each of the two piles
* Repeat until every pile only contains things that do/don't fly
* Somewhat similar to playing a game of "20 questions"

### Reflection
* _What was the first characteristic you used to divide the pictures? Why?_
* _What were the second characteristics you used to divide the pictures? Were they the same for each pile? Why or why not?_
* _Overall, how did you choose questions?_
* _What answer would you give for a previously unseen thing—e.g., a train? a pigeon? a kite?_

## scikit-learn

* A collection of Python modules for performing machine learning
* High-level steps
    * 1) Load data — reuse functions we wrote on November 12
    * 2) Train — called `fit` in scikit-learn
    * 3) Predict
    
### Load data
* Assume we have a comma-separated variable file with a row for each of the items in the prior activity and columns for the label and features
```
object,flies,has_wings,has_propeller,num_legs,num_wheels
penguin,doesn't fly,1,0,2,0
ostrich,doesn't fly,1,0,2,0
truck,doesn't fly,0,0,0,6
car,doesn't fly,0,0,0,4
caterpillar,doesn't fly,0,0,8,0
ant,doesn't fly,0,0,6,0
dog,doesn't fly,0,0,4,0
cat,doesn't fly,0,0,4,0
biplane,flies,1,0,0,2
jetplane,flies,1,0,0,3
butterfly,flies,1,0,4,0
ladybug,flies,2,0,4,0
eagle,flies,1,0,2,0
hummingbird,flies,1,0,0,0
helicopter,flies,0,1,0,0
drone,flies,0,1,0,0
```

In [46]:
with open('things.csv', 'w') as datafile:
    datafile.write("object,flies,has_wings,has_propeller,num_legs,num_wheels\npenguin,doesn't fly,1,0,2,0\nostrich,doesn't fly,1,0,2,0\ntruck,doesn't fly,0,0,0,6\ncar,doesn't fly,0,0,0,4\ncaterpillar,doesn't fly,0,0,8,0\nant,doesn't fly,0,0,6,0\ndog,doesn't fly,0,0,4,0\ncat,doesn't fly,0,0,4,0\nbiplane,flies,1,0,0,2\njetplane,flies,1,0,0,3\nbutterfly,flies,1,0,4,0\nladybug,flies,2,0,4,0\neagle,flies,1,0,2,0\nhummingbird,flies,1,0,0,0\nhelicopter,flies,0,1,0,0\ndrone,flies,0,1,0,0")

* Slightly modified `load_data` function from November 12

In [24]:
def load_data(filename, names):
    '''
    Loads data from a CSV file

    >>> load_data('sample.csv', [])
    [['Apricot', 'Orange', 'Fruit'], ['Blueberry', 'Blue', 'Fruit'], ['Carrot', 'Orange', 'Vegetable']]
    '''
    file = open(filename, 'r')
    data = []
    # Get column names
    names += file.readline().strip().split(',')
    # Process each row
    for line in file:
        # Get values for each column
        line = line.strip().split(',')
        data.append(line)
    file.close()
    return data

* _Which column contains the labels?_ — 2nd column
* _Which column contains features?_ — 3rd through 6th columns
* Helper functions for selecting columns from November 12

In [25]:
def select_single_column(data, idx):
    '''
    Extract a single column of values from a "table" of data
    
    >>> select_single_column([['Apricot','Orange','Fruit'], ['Blueberry','Blue','Fruit'], ['Carrot','Orange','Vegetable']], 0)
    ['Apricot', 'Blueberry', 'Carrot']
    '''
    values = []
    # Process each row
    for row in data:
        values.append(row[idx])
    return values

def select_columns(data, start, end):
    '''
    Extract a range of columns from a "table" of data
    
    >>> select_columns([['Apricot','Orange','Fruit'], ['Blueberry','Blue','Fruit'], ['Carrot','Orange','Vegetable']], 0, 2)
    [['Apricot', 'Orange'], ['Blueberry', 'Blue'], ['Carrot', 'Orange']]
    '''
    filtered = []
    # Process each row
    for row in data:
        filtered.append(row[start:end])
    return filtered


* Data loading portion of main

In [31]:
column_names = []
data = load_data('things.csv', column_names)
labels = select_single_column(data, 1)
features = select_columns(data,2,6)

### Training
* Use `tree` submodule from `sklearn` module
    * _What did we do when we used the `turtle` or `random` modules?_ — `import`

In [32]:
from sklearn import tree

* Create a `DecisionTreeClassifier` object
    * _How did we create a `Turtle` object?_ — `turtle.Turtle()`

In [33]:
dtree = tree.DecisionTreeClassifier()

* Train the `DecisionTreeClassifier` using the `fit` method
    * _How did we make a `Turtle` move?_ — turtle.forward(100)

In [34]:
dtree.fit(features, labels)

DecisionTreeClassifier()

* Display the tree using the `export_tree` function

In [35]:
print(tree.export_text(dtree, feature_names=column_names[2:6]))

|--- has_wings <= 0.50
|   |--- has_propeller <= 0.50
|   |   |--- class: doesn't fly
|   |--- has_propeller >  0.50
|   |   |--- class: flies
|--- has_wings >  0.50
|   |--- num_legs <= 1.00
|   |   |--- class: flies
|   |--- num_legs >  1.00
|   |   |--- num_legs <= 3.00
|   |   |   |--- class: doesn't fly
|   |   |--- num_legs >  3.00
|   |   |   |--- class: flies



### Prediction
* Create a list of features (`has_wings`, `has_propeller`, `num_legs`, `num_wheels`)

In [38]:
train_features = [0, 0, 0, 8]

* Use the `DecisionTreeClassifier`'s `predict` method
    * Takes a list of lists, where each sublist is a single set of features
    * Returns a list of predictions, where there is one prediction for each sublist provided as a parameter

In [40]:
predict_features = [train_features]
predictions = dtree.predict(predict_features)
print(predictions[0])

doesn't fly


* More predictions

In [44]:
pigeon_features = [1, 0, 2, 0]
kite_features = [0, 0, 0, 0]
predict_features = [pigeon_features, kite_features]
predictions = dtree.predict(predict_features)
print("Pigeon", predictions[0])
print("Kite", predictions[1])

Pigeon doesn't fly
Kite doesn't fly


## Accuracy
* _Why aren't the predictions correct?_
    * Not enough features
    * Too few training samples
    * Biased training data
* Add a more examples of things that fly, then re-train
```
robin,flies,1,0,2,0
bluejay,flies,1,0,2,0
cardinal,flies,1,0,2,0
fly,flies,1,0,6,0
flag,flies,0,0,0,0
```

In [ ]:
with open('things.csv', 'w') as datafile:
        datafile.write("object,flies,has_wings,has_propeller,num_legs,num_wheels\npenguin,doesn't fly,1,0,2,0\nostrich,doesn't fly,1,0,2,0\ntruck,doesn't fly,0,0,0,6\ncar,doesn't fly,0,0,0,4\ncaterpillar,doesn't fly,0,0,8,0\nant,doesn't fly,0,0,6,0\ndog,doesn't fly,0,0,4,0\ncat,doesn't fly,0,0,4,0\nbiplane,flies,1,0,0,2\njetplane,flies,1,0,0,3\nbutterfly,flies,1,0,4,0\nladybug,flies,2,0,4,0\neagle,flies,1,0,2,0\nhummingbird,flies,1,0,0,0\nhelicopter,flies,0,1,0,0\ndrone,flies,0,1,0,0\nrobin,flies,1,0,2,0\nbluejay,flies,1,0,2,0\ncardinal,flies,1,0,2,0\nfly,flies,1,0,6,0\nflag,flies,0,0,0,0")

In [50]:
column_names = []
data = load_data('things.csv', column_names)
labels = select_single_column(data, 1)
features = select_columns(data,2,6)

dtree = tree.DecisionTreeClassifier()
dtree.fit(features, labels)
print(tree.export_text(dtree, feature_names=column_names[2:6]))

train_features = [0, 0, 0, 8]
pigeon_features = [1, 0, 2, 0]
kite_features = [0, 0, 0, 0]
penguin_features = [1,0,2,0]
predict_features = [train_features, pigeon_features, kite_features, penguin_features]
predictions = dtree.predict(predict_features)
print("Train", predictions[0])
print("Pigeon", predictions[1])
print("Kite", predictions[2])
print("Penguin", predictions[3])

|--- has_wings <= 0.50
|   |--- has_propeller <= 0.50
|   |   |--- num_legs <= 2.00
|   |   |   |--- num_wheels <= 2.00
|   |   |   |   |--- class: flies
|   |   |   |--- num_wheels >  2.00
|   |   |   |   |--- class: doesn't fly
|   |   |--- num_legs >  2.00
|   |   |   |--- class: doesn't fly
|   |--- has_propeller >  0.50
|   |   |--- class: flies
|--- has_wings >  0.50
|   |--- num_legs <= 1.00
|   |   |--- class: flies
|   |--- num_legs >  1.00
|   |   |--- num_legs <= 3.00
|   |   |   |--- class: flies
|   |   |--- num_legs >  3.00
|   |   |   |--- class: flies

Train doesn't fly
Pigeon flies
Kite flies
Penguin flies


* Model may be _overfit_ to training data
    * Use an ensemble of decision trees — i.e., a random forest

## Random forest
* Train multiple decision trees
* Each tree is trained on a different subset of the training data
* Make a prediction with every tree, and use the most common prediction as the final prediction

### Activity
* Add additional pictures into training set
* Randomly remove 8 pictures
* Manually construct a decision tree to decide _does it fly?_
* After each group is done, every tree will make a prediction and we'll choose the majority vote
* Things to predict
    * Train
    * Pigeon
    * Kite
    * Penguin